In [1]:
from IPython import parallel
c = parallel.Client()
view = c.load_balanced_view()

In [2]:
%%px --local

import numpy as np
from scipy import ndimage
from scipy import misc
from scipy import linalg
import os
import glob
import h5py as h5

In [3]:
%%px --local
class FiberAngles:
    def __init__(self, inputfilename, outputfilename, volsize):
        self.inname = inputfilename
        f = h5.File(self.inname,'r')
        self.imgdata = f['image']

        self.outname = outputfilename
        self.h5outfile = h5.File(self.outname,'a')
        
        #probably there's a better way to do this, but this works
        assert(len(volsize) == len(self.imgdata.shape))
        self.volsz = volsize
        
    def hessian_ctr3(self, di, C):
        a1 = np.array([-1.0, 9.0, -45.0, 0.0, 45.0, -9.0, 1.0])/60
        a2 = np.array([2, -27, 270, -490, 270, -27, 2])/180.0

        #these both automatically round down, which is what we want
        n = len(a1)/2
        c = np.array(C.shape)/2
        c = c[0]

        ic = np.arange(c-n,c+n+1)

        H = np.zeros((3,3))
        H[0,0] = sum(C[ic,c,c] * a2) / di**2
        H[1,1] = sum(C[c,ic,c] * a2) / di**2
        H[2,2] = sum(C[c,c,ic] * a2) / di**2

        ic1 = ic[:,np.newaxis]
        ic2 = ic[np.newaxis,:]

        H[0,1] = np.sum( np.sum( C[ic1,ic2,c] * a1[:,np.newaxis], axis=0) * a1) / di**2
        H[0,2] = np.sum( np.sum( C[ic1,c,ic2] * a1[:,np.newaxis], axis=0) * a1) / di**2
        H[1,2] = np.sum( np.sum( C[c,ic1,ic2] * a1[:,np.newaxis], axis=0) * a1) / di**2

        H[1,0] = H[0,1]
        H[2,0] = H[0,2]
        H[2,1] = H[1,2]

        return H

    def get_angle3(self, C):
        H = self.hessian_ctr3(1,C)
        w,vr = linalg.eig(H)

        ord = np.argsort(np.abs(w))
        vr = vr[:,ord]
        w = w[ord]
        
        return vr, w
    
    def get_angle_from_file(self, ctr):
        volsz2 = [int(np.floor(vs/2)) for vs in self.volsz]
        sl = tuple([slice(i-vs2,i+vs2) for (i,vs2) in zip(ctr,volsz2)])
        
        V = self.imgdata[sl]
        
        return self.get_angle3(V)
    
    def setup_out_file(self, overlap=0):
        step = np.multiply(self.volsz, 1-overlap)
        
        i = [np.arange(size1/2,len1-size1/2,step1) for size1,len1,step1 \
             in zip(self.volsz,self.imgdata.shape,step)]
        self.grid = np.meshgrid(*i,indexing='ij')
        
        sh = self.grid[0].shape

        #save the grid
        griddataset = self.h5outfile.require_dataset('grid',(3,sh[0],sh[1],sh[2]), dtype=np.int32, exact=True)
        for j,grid1 in enumerate(self.grid):
            griddataset[j,:,:,:] = grid1
        
        #set up to save eigenvectors and eigenvalues
        self.eigvecdata = self.h5outfile.require_dataset('eigenvectors',(3,3,sh[0],sh[1],sh[2]),\
                                      chunks=(3,1,4,4,4), dtype=np.double, exact=True)
        self.eigvaldata = self.h5outfile.require_dataset('eigenvalues',(3,1,sh[0],sh[1],sh[2]),\
                                      chunks=(3,1,4,4,4), dtype=np.double, exact=True)
    
    def open_out_file(self):
        grid = self.h5outfile['grid']
        self.grid = [grid[i] for i in np.arange(0,3)]
        sh = self.grid[0].shape
        
        #set up to save eigenvectors and eigenvalues
        self.eigvecdata = self.h5outfile.require_dataset('eigenvectors',(3,3,sh[0],sh[1],sh[2]),\
                                      chunks=(3,1,4,4,4), dtype=np.double, exact=True)
        self.eigvaldata = self.h5outfile.require_dataset('eigenvalues',(3,1,sh[0],sh[1],sh[2]),\
                                      chunks=(3,1,4,4,4), dtype=np.double, exact=True)

        
    def get_angle_and_save(self, gridind):
        ctr = [g[gridind[0],gridind[1],gridind[2]] for g in self.grid]
        
        vr, w = self.get_angle_from_file(ctr)
        
        self.eigvecdata[:,:,gridind[0],gridind[1],gridind[2]] = vr
        self.eigvaldata[:,0,gridind[0],gridind[1],gridind[2]] = w.real

In [4]:
%%px --local
def run_angle_comp(gridind):
    F.get_angle_and_save(gridind)

In [5]:
F = FiberAngles('Drerio4.h5','Drerio4angles1.h5',[32,32,32])
F.setup_out_file()

In [7]:
%%px
Frem = FiberAngles('Drerio4.h5','Drerio4angles1.h5',[32,32,32])
Frem.open_out_file()

CompositeError: one or more exceptions from call to method: execute
[0:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"
[1:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"
[2:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"
[3:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"

In [48]:
F2.open_out_file()

In [46]:
%px F.open_out_file()

CompositeError: one or more exceptions from call to method: execute
[0:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"
[1:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"
[2:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"
[3:execute]: KeyError: "Unable to open object (Object 'grid' doesn't exist)"

In [14]:
sh = F.grid[0].shape
indices = np.meshgrid(range(0,sh[0]),range(0,sh[1]),range(0,sh[2]),indexing='ij')

In [16]:
indices = zip(*map(np.ndarray.flatten,indices))

In [21]:
indices[22000:22010]

[(8, 14, 58),
 (8, 14, 59),
 (8, 14, 60),
 (8, 14, 61),
 (8, 14, 62),
 (8, 14, 63),
 (8, 14, 64),
 (8, 14, 65),
 (8, 14, 66),
 (8, 14, 67)]

In [22]:
view.map_sync(run_angle_comp,indices[22000:22010])

CompositeError: one or more exceptions from call to method: run_angle_comp
[1:apply]: TypeError: slice indices must be integers or None or have an __index__ method
[3:apply]: TypeError: slice indices must be integers or None or have an __index__ method
[0:apply]: TypeError: slice indices must be integers or None or have an __index__ method
[2:apply]: TypeError: slice indices must be integers or None or have an __index__ method
.... 6 more exceptions ...